### 과제
#### 1. 위의 누적수익률 그래프에서 잘못된 점을 찾아 이를 바로 잡는 코드를 작성하시오.
#### 2. 위의 DD 그래프에서 잘못된 점을 찾아 이를 바로 잡는 코드를 작성하시오.**

In [2]:
import FinanceDataReader as fdr
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
def getCloseDataPandas(ticker, start,end=None):
    #print("ticker: " ,ticker)
    return data.get_data_yahoo(ticker, start,end)['Close']

def getCloseData(ticker, start, end=None):
    #print("ticker: " ,ticker)
    return fdr.DataReader(ticker,start,end)['Close']

def getDayReturn(closeDataSet):
    return (closeDataSet/closeDataSet.shift(1)).fillna(1)

def getCumulativeReturn(closeDataSet):
    return closeDataSet / closeDataSet.iloc[0]

def getPortfolioResult(closeDataSet, weight = None):
    dayReturn = getDayReturn(closeDataSet)
    cumulativeReturn = getCumulativeReturn(closeDataSet)
    if not weight:
        weight = [1/len(closeDataSet.columns)] * len(closeDataSet.columns)
        
    portfolioCumulativeReturn = (weight*cumulativeReturn).sum(axis=1)
    portfolioDayReturn = (portfolioCumulativeReturn/portfolioCumulativeReturn.shift(1)).fillna(1)
    
    return portfolioDayReturn, portfolioCumulativeReturn

def getEvaluation(cumulativeReturn):
    cagr = cumulativeReturn.iloc[-1] ** (252/len(cumulativeReturn))
    dd = (cumulativeReturn.cummax() - cumulativeReturn)/ cumulativeReturn.cummax()*100
    
    plt.figure(figsize=(15,5))
    plt.plot(-dd, label="DrawDown")
    plt.legend(fontsize = 16)
    mdd = dd.max()
    
    print(f"최종 수익률 : {cumulativeReturn.iloc[-1]}\ncagr: {cagr}\nmdd: {mdd}")
    return cagr,dd,mdd


def getRebalancingDate(closeDataSet, period="month"):
    data = closeDataSet.copy()
    data = pd.DataFrame(data) 
    data.index = pd.to_datetime(data.index)
    data['year'] = data.index.year
    data['month'] = data.index.month
    if period == "month":
        rebalancingDate = data.drop_duplicates(['year', 'month'], keep="last").index
    if period == "quarter":
        quarter = [3,6,9,12]
        data = data.loc[data['month'].isin(quarter)]
        rebalancingDate = data.drop_duplicates(['year', 'month'], keep="last").index
    if period == "year":
        rebalancingDate = data.drop_duplicates(['year'], keep="last").index
    return rebalancingDate

def getRebalancingPortfolioResult(closeDataSet, period = "month", weightDf=None):
    if weightDf is None:
        rebalancingDate = getRebalancingDate(closeDataSet, period) 
        weightDf = pd.DataFrame([[1/len(closeDataSet.columns)] * len(closeDataSet.columns)] * len(rebalancingDate),
                                index=rebalancingDate,
                                columns=closeDataSet.columns)
    else:
        closeDataSet = closeDataSet.loc[weightDf.index[0]:]
        rebalancingDate = getRebalancingDate(closeDataSet, period) 
    portfolio = pd.DataFrame()
    totalAsset = 1 
    start = rebalancingDate[0] 
    for end in rebalancingDate[1:]:
        weight = weightDf.loc[start]
        priceData = closeDataSet.loc[start:end] 
        cumReturn = getCumulativeReturn(priceData) 
        weightedCumReturn = weight * cumReturn 
        netCumReturn = totalAsset * weightedCumReturn
        start = end # start 갱신
        totalAsset = netCumReturn.iloc[-1].sum()
        portfolio = pd.concat([portfolio, netCumReturn])
    portfolio = portfolio.loc[~portfolio.index.duplicated(keep='last')] 
    portfolioCumulativeReturn = portfolio.sum(axis=1)
    portfolioDayReturn = (portfolioCumulativeReturn / portfolioCumulativeReturn.shift(1)).fillna(1)
    
    return portfolioDayReturn, portfolioCumulativeReturn

def getDay(str0):
    SY = int(str0[:4])
    SM = int(str0[5:7])
    SD = int(str0[8:10])
    return SY,SM,SD 


# 매월 마지막날 종가를 기준으로 모메텀 계산

# 1. 주식

In [4]:
kodex200 = getCloseData("069500", "2021-9-1","2022-9-30")
kodex200

Date
2021-09-01    41200
2021-09-02    40823
2021-09-03    41087
2021-09-06    41230
2021-09-07    40984
              ...  
2022-09-26    29050
2022-09-27    29105
2022-09-28    28390
2022-09-29    28400
2022-09-30    28275
Name: Close, Length: 266, dtype: int64

In [5]:
rebalDate = getRebalancingDate(kodex200)
rebalDate

DatetimeIndex(['2021-09-30', '2021-10-29', '2021-11-30', '2021-12-30',
               '2022-01-28', '2022-02-28', '2022-03-31', '2022-04-29',
               '2022-05-31', '2022-06-30', '2022-07-29', '2022-08-31',
               '2022-09-30'],
              dtype='datetime64[ns]', name='Date', freq=None)

In [6]:
priceOnRebalDate =  kodex200.loc[rebalDate]
priceOnRebalDate

Date
2021-09-30    39387
2021-10-29    38126
2021-11-30    36757
2021-12-30    39153
2022-01-28    35656
2022-02-28    36005
2022-03-31    36472
2022-04-29    35433
2022-05-31    35363
2022-06-30    30808
2022-07-29    32385
2022-08-31    32230
2022-09-30    28275
Name: Close, dtype: int64

In [20]:
str(rebalDate[-1])

'2022-09-30 00:00:00'

In [7]:
SY,SM,SD = getDay(str(rebalDate[-1]))
n=12
mY = SY
mM = SM

if SM-n <1:
    mY = SY-1
    mM = 12+SM -n

standardDay = str(rebalDate[-1])
newDay=str(mY)+"-"+str(mM).zfill(2)

In [8]:
standardDay

'2022-09-30 00:00:00'

In [9]:
newDay

'2021-09'

In [10]:
this_month = int(priceOnRebalDate[standardDay])

In [11]:
last_n_month = int(priceOnRebalDate[newDay])

In [12]:
print(this_month/last_n_month)

0.7178764566989108


# 2. 채권

In [237]:
kodex10Bond = getCloseData("152380", "2021-9-1","2022-9-30")
kodex10Bond

Date
2021-09-01    70455
2021-09-02    70450
2021-09-03    70425
2021-09-06    70295
2021-09-07    70270
              ...  
2022-09-26    59390
2022-09-27    59915
2022-09-28    59775
2022-09-29    59770
2022-09-30    60035
Name: Close, Length: 266, dtype: int64

In [238]:
rebalDate = getRebalancingDate(kodex10Bond)
rebalDate

DatetimeIndex(['2021-09-30', '2021-10-29', '2021-11-30', '2021-12-30',
               '2022-01-28', '2022-02-28', '2022-03-31', '2022-04-29',
               '2022-05-31', '2022-06-30', '2022-07-29', '2022-08-31',
               '2022-09-30'],
              dtype='datetime64[ns]', name='Date', freq=None)

In [239]:
priceOnRebalDate =  kodex10Bond.loc[rebalDate]
priceOnRebalDate

Date
2021-09-30    68745
2021-10-29    69400
2021-11-30    68970
2021-12-30    68840
2022-01-28    66970
2022-02-28    66685
2022-03-31    65130
2022-04-29    63870
2022-05-31    63640
2022-06-30    62025
2022-07-29    64915
2022-08-31    61935
2022-09-30    60035
Name: Close, dtype: int64

In [240]:
SY,SM,SD = getDay(str(rebalDate[-1]))
n=12
mY = SY
mM = SM

if SM-n <1:
    mY = SY-1
    mM = 12+SM -n

standardDay = str(rebalDate[-1])
newDay=str(mY)+"-"+str(mM).zfill(2)

In [241]:
standardDay

'2022-09-30 00:00:00'

In [242]:
newDay

'2021-09'

In [243]:
this_month = int(priceOnRebalDate[standardDay])

In [244]:
last_n_month = int(priceOnRebalDate[newDay])

In [245]:
print(this_month/last_n_month)

0.873299876354644


# 함수화1

In [21]:
def getMomentumScore(ticker, start, end,n):
    data = getCloseData(ticker,start,end)
    rebalDate = getRebalancingDate(data)
    priceOnRebalDate =  data.loc[rebalDate]
    
    SY,SM,SD = getDay(str(rebalDate[-1]))

    mY = SY
    mM = SM

    if SM-n <1:
        mY = SY-1
        mM = 12+SM -n
    else :
        mM = SM-n

    standardDay =str(SY)+"-"+str(SM).zfill(2)+"-"+str(SD).zfill(2)
    newDay=str(mY)+"-"+str(mM).zfill(2)

    this_month = int(priceOnRebalDate[standardDay])
    last_n_month = int(priceOnRebalDate[newDay])
   
    return standardDay,newDay,this_month/last_n_month

In [22]:
# 주식 
getMomentumScore("069500", "2021-9-1","2022-9-30",12)

('2022-09-30', '2021-09', 0.7178764566989108)

In [23]:
# 채권
getMomentumScore("152380", "2021-9-1","2022-9-30",12)

('2022-09-30', '2021-09', 0.873299876354644)

In [24]:
getMomentumScore("069500", "2021-9-1","2022-9-30",1)

('2022-09-30', '2022-08', 0.8772882407694694)

# 평균 모멘텀

In [45]:
kodex200 =[]
kodex10Bond =[]

n =12

for i in range(1, n+1):
    Standard,New,a= getMomentumScore("069500", "2020-1-2","2022-07-29",i)
    Standard,New,b= getMomentumScore("152380",  "2020-1-2","2022-07-29",i)
    
    print(i,"모멘텀 스코어 ",Standard,"/",New," : ", a,b)
    print("")
    kodex200.append(int(a))
    kodex10Bond.append(int(b))
    
print(sum(kodex200)/n , sum(kodex10Bond)/n)


1 모멘텀 스코어  2022-07-29 / 2022-06  :  1.0511880031160739 1.0465941152760982

2 모멘텀 스코어  2022-07-29 / 2022-05  :  0.9157876876961796 1.020034569453174

3 모멘텀 스코어  2022-07-29 / 2022-04  :  0.9139784946236559 1.01636135901049

4 모멘텀 스코어  2022-07-29 / 2022-03  :  0.8879414345251152 0.9966989098725626

5 모멘텀 스코어  2022-07-29 / 2022-02  :  0.8994584085543674 0.9734572992427083

6 모멘텀 스코어  2022-07-29 / 2022-01  :  0.9082622840475656 0.9693146184858892

7 모멘텀 스코어  2022-07-29 / 2021-12  :  0.8271396827829285 0.9429837303893085

8 모멘텀 스코어  2022-07-29 / 2021-11  :  0.8810566694779226 0.9412063215890967

9 모멘텀 스코어  2022-07-29 / 2021-10  :  0.8494203430729685 0.9353746397694525

10 모멘텀 스코어  2022-07-29 / 2021-09  :  0.8222256074339249 0.94428685722598

11 모멘텀 스코어  2022-07-29 / 2021-08  :  0.7871709486886561 0.9186951599207472

12 모멘텀 스코어  2022-07-29 / 2021-07  :  0.7779058874396484 0.9176562058241448

0.08333333333333333 0.25


In [29]:
kodex200

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [30]:
kodex10Bond

[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [40]:
sum(kodex200)/12

0.08333333333333333

In [44]:
len(kodex200)

12

# 함수화2

In [46]:
def getMomentumScore(ticker, start, end,n):
    data = getCloseData(ticker,start,end)
    rebalDate = getRebalancingDate(data)
    priceOnRebalDate =  data.loc[rebalDate]
    
    SY,SM,SD = getDay(str(rebalDate[-1]))

    mY = SY
    mM = SM

    if SM-n <1:
        mY = SY-1
        mM = 12+SM -n
    else :
        mM = SM-n

    standardDay =str(SY)+"-"+str(SM).zfill(2)+"-"+str(SD).zfill(2)
    newDay=str(mY)+"-"+str(mM).zfill(2)

    this_month = int(priceOnRebalDate[standardDay])
    last_n_month = int(priceOnRebalDate[newDay])
   
    return standardDay,newDay,this_month/last_n_month

In [47]:
def getAvgMomentumScore(ticker, start, end, n):
    li =[]

    for i in range(1, n+1):
        Standard,New,a= getMomentumScore(ticker, start,end,i)

        #print(i,"모멘텀 스코어 ",Standard,"/",New," : ", a,b)
        #print("")
        li.append(int(a))

    return sum(li)/n 


In [49]:
getAvgMomentumScore("152380", "2020-1-2","2022-07-29",12)

0.25

# test

In [50]:
getAvgMomentumScore("069500", "2020", "2022-7-30",12)

0.08333333333333333